In [1]:
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau


torch.manual_seed(41)
np.random.seed(41)



In [2]:

class KFoldDataset(Dataset):
    def __init__(self, data_files, target_files):
        self.data = []
        self.targets = []
        
        for data_file, target_file in zip(data_files, target_files):
            data = np.load(data_file)
            targets = np.load(target_file)
            
            self.data.append(data)
            self.targets.append(targets)
    
    def __getitem__(self, index):
        fold_idx = index // len(self.data[0])
        item_idx = index % len(self.data[0])
        
        data = self.data[fold_idx][item_idx]
        target = self.targets[fold_idx][item_idx]
        
        return torch.from_numpy(data), torch.from_numpy(target)
    
    def __len__(self):
        return len(self.data) * len(self.data[0])

In [3]:
from torch.utils.data import DataLoader

# Define the base file name and number of folds
# base_file_name = 'jet_images_c8_minpt2_allfeats_robust_fast'
base_file_name = 'jet_images_c8_minpt2_ptetaphi_robust_fast'
num_folds = 5

def datamaker(base_file_name, num_folds, val_fold, batch_size=32):
    # Generate the file paths for your k-fold data and targets
    train_folds = [i for i in range(num_folds) if i != val_fold]
    val_folds = [val_fold]
    train_data_files = [f'./normalized_data2/x_{base_file_name}_{i}.npy' for i in train_folds]
    train_target_files = [f'./normalized_data2/y_{base_file_name}_{i}.npy' for i in train_folds]
    val_data_files = [f'./normalized_data2/x_{base_file_name}_{i}.npy' for i in val_folds]
    val_target_files = [f'./normalized_data2/y_{base_file_name}_{i}.npy' for i in val_folds]
    
    train_dataset = KFoldDataset(train_data_files, train_target_files)
    val_dataset = KFoldDataset(val_data_files, val_target_files)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

    return train_loader, val_loader




train_loader, val_loader = datamaker(base_file_name, num_folds, 3)

In [4]:
import torch
import torch.nn as nn

class DeepSetsInv(nn.Module):
    def __init__(self, input_size, nnodes_phi: int = 32, nnodes_rho: int = 16, activ: str = "relu"):
        super(DeepSetsInv, self).__init__()
        self.nclasses = 5
        
        self.phi = nn.Sequential(
            nn.Linear(input_size, nnodes_phi),
            self.get_activation(activ),
            nn.Linear(nnodes_phi, nnodes_phi),
            self.get_activation(activ),
            nn.Linear(nnodes_phi, nnodes_phi),
            self.get_activation(activ),
        )
        
        self.rho = nn.Sequential(
            nn.Linear(nnodes_phi, nnodes_rho),
            self.get_activation(activ),
            nn.Linear(nnodes_rho, self.nclasses),
        )
    
    def get_activation(self, activ):
        if activ == "relu":
            return nn.ReLU()
        elif activ == "sigmoid":
            return nn.Sigmoid()
        elif activ == "tanh":
            return nn.Tanh()
        else:
            raise ValueError(f"Unsupported activation: {activ}")
    
    def forward(self, inputs):
        phi_output = self.phi(inputs)
        # print("phi_output dtype:", phi_output.dtype)
        sum_output = torch.mean(phi_output, dim=1)
        # print( "sum_output dtype:", sum_output.dtype)
        rho_output = self.rho(sum_output)
        # print( "rho_output dtype:", rho_output.dtype)
        return rho_output

In [5]:
# Create the model
input_size = 3  # Assuming each input feature vector has a size of 16
model = DeepSetsInv(input_size=input_size, nnodes_phi=32, nnodes_rho=16, activ="relu")

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0032)
lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 100
model.to(device)

# Training loop
best_val_accuracy = 0.0
patience= 7
for epoch in range(num_epochs):
    lossval = 0.0
    model.train() 
    for batch_data, batch_targets in train_loader:
        batch_data = batch_data.to(device).float()
        batch_targets = batch_targets.to(device).float()
        
        # Forward pass
        outputs = model(batch_data)
        
        loss = criterion(outputs, batch_targets)
        lossval += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()  
    with torch.no_grad():
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        for val_data, val_targets in val_loader:
            val_data = val_data.to(device).float()
            val_targets = val_targets.to(device).float()
            
            outputs = model(val_data)
            loss = criterion(outputs, val_targets)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            true_labels = torch.argmax(val_targets, 1)  # Get the true class labels
            
            val_total += true_labels.size(0)
            val_correct += (predicted == true_labels).sum().item()

        val_accuracy = val_correct / val_total
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter > patience:
                print(f"Early stopping: Validation accuracy has not increased in {patience} epochs")
                break
        lr_scheduler.step(val_accuracy)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {lossval/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

Epoch 1/100, Train Loss: 1.0886, Val Loss: 1.0552, Val Accuracy: 0.5899
Epoch 2/100, Train Loss: 1.0435, Val Loss: 1.0323, Val Accuracy: 0.6050
Epoch 3/100, Train Loss: 1.0352, Val Loss: 1.0361, Val Accuracy: 0.6005
Epoch 4/100, Train Loss: 1.0317, Val Loss: 1.0395, Val Accuracy: 0.6036
Epoch 5/100, Train Loss: 1.0296, Val Loss: 1.0422, Val Accuracy: 0.5943
Epoch 6/100, Train Loss: 1.0278, Val Loss: 1.0277, Val Accuracy: 0.6098
Epoch 7/100, Train Loss: 1.0266, Val Loss: 1.0216, Val Accuracy: 0.6092
Epoch 8/100, Train Loss: 1.0257, Val Loss: 1.0285, Val Accuracy: 0.6070
Epoch 9/100, Train Loss: 1.0248, Val Loss: 1.0367, Val Accuracy: 0.6061
Epoch 10/100, Train Loss: 1.0243, Val Loss: 1.0202, Val Accuracy: 0.6115
Epoch 11/100, Train Loss: 1.0241, Val Loss: 1.0190, Val Accuracy: 0.6108
Epoch 12/100, Train Loss: 1.0234, Val Loss: 1.0298, Val Accuracy: 0.6014
Epoch 13/100, Train Loss: 1.0230, Val Loss: 1.0531, Val Accuracy: 0.5881
Epoch 14/100, Train Loss: 1.0230, Val Loss: 1.0375, Val Accu

In [6]:
def test_loader_maker(base_file_name, batch_size=32):
    # /Users/dimademler/Documents/GitHub/Morph-DeepSets/normalized_data/y_jet_images_c8_minpt2_allfeats_robust_fast_test.npy
    test_data_file = f'./normalized_data2/x_{base_file_name}_test.npy'
    test_target_file = f'./normalized_data2/y_{base_file_name}_test.npy'

    test_dataset = KFoldDataset([test_data_file], [test_target_file])
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return test_loader


def test_model(model, test_loader):
    model.eval()
    with torch.no_grad():
        test_correct = 0
        test_total = 0
        for test_data, test_targets in test_loader:
            test_data = test_data.to(device).float()
            test_targets = test_targets.to(device).float()
            
            outputs = model(test_data)
            _, predicted = torch.max(outputs, 1)
            true_labels = torch.argmax(test_targets, 1)  # Get the true class labels
            
            test_total += true_labels.size(0)
            test_correct += (predicted == true_labels).sum().item()
        
        test_accuracy = test_correct / test_total
        print(f"Test Accuracy: {test_accuracy:.4f}")

test_loader = test_loader_maker(base_file_name)
test_model(model, test_loader)


Test Accuracy: 0.6163
